# Install Dependencies

## External Dependencies

In [1]:
!pip install inflect


[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


## FrostAura Dependencies

In [2]:
import os
import sys

USE_LOCAL_CODE: bool = True # NOTE: When changing this configuration, be sure to restart the interpreter in order to have sys path updates reflect.
directories_to_move_up: int = 2
current_executing_path: str = os.getcwd()
current_executing_path = current_executing_path.replace('\\', '/')
root_path: str = '/'.join(current_executing_path.split('/')[:-directories_to_move_up])

# Clean up.
%pip uninstall frostaura -y
sys.path = [p for p in sys.path if p != root_path]

if USE_LOCAL_CODE:
    sys.path.append(root_path)
else:
    %pip install -U --no-cache-dir frostaura

Note: you may need to restart the kernel to use updated packages.


# Import Dependencies

## Logging

In [3]:
import logging

logging.getLogger().disabled = True

## Imports

In [4]:
from frostaura.engines import IAssetProjectionEngine
from frostaura.engines import SimpleAssetProjectionEngine
import pandas as pd
from frostaura.engines import IAssetValuationEngine
from frostaura.engines import GrahamValuationEngine
from frostaura.engines import IAssetCalculationsEngine
from frostaura.engines import SimpleAssetCalculationsEngine
from frostaura.data_access import EasyEquitiesPersonalAssetDataAccess
from frostaura.data_access import IResourcesDataAccess
from frostaura.data_access import EmbeddedResourcesDataAccess
from frostaura.data_access import HtmlResourcesDataAccess
from frostaura.data_access import IPersonalAssetDataAccess
from frostaura.data_access import IPublicAssetDataAccess
from frostaura.data_access import YahooFinanceDataAccess
from frostaura.models import ValuationResult

# Usage

## Setup

In [5]:
html_data_access: IResourcesDataAccess = HtmlResourcesDataAccess()
resource_data_access: IResourcesDataAccess = EmbeddedResourcesDataAccess()
public_asset_data_access: IPublicAssetDataAccess = YahooFinanceDataAccess()
asset_valuation_engine: IAssetValuationEngine = GrahamValuationEngine(html_data_access=html_data_access, public_asset_data_access=public_asset_data_access)
asset_calculations_engine: IAssetCalculationsEngine = SimpleAssetCalculationsEngine(public_asset_data_access=public_asset_data_access)
personal_asset_data_access: IPersonalAssetDataAccess = EasyEquitiesPersonalAssetDataAccess(resource_data_access=resource_data_access,
                                                                         username=None,
                                                                         password=None)
instance: IAssetProjectionEngine = SimpleAssetProjectionEngine(asset_valuation_engine=asset_valuation_engine,
                                                               public_asset_data_access=public_asset_data_access)

## Project for a Single Symbol

In [6]:
n_months: int = 12*30 # 39 year
usd_zar_exchange_rate: float = 16.17
symbol: str = 'AAPL'
holdings: pd.DataFrame = personal_asset_data_access.get_personal_transactions()
holdings_with_profits: pd.DataFrame = asset_calculations_engine.calculate_holdings_profits(holdings=holdings)
initial_value: float = holdings_with_profits.loc[holdings_with_profits['symbol'] == 'AAPL']['total_current_usd'].sum()
monthly_deposit: float = 2000 / usd_zar_exchange_rate / len(holdings_with_profits)

growth: pd.DataFrame = instance.project_monthly_asset_growth(n_months=n_months,
                                                             symbol=symbol,
                                                             principal_value=initial_value,
                                                             monthly_deposit=monthly_deposit)

growth

,deposits_withdrawals,interest,total_deposits_withdrawals,accrued_interest,balance
month,,,,,
0,0.000000,0.000000,0.000000,0.000000,66.481800
1,24.737168,0.525206,24.737168,0.525206,91.744174
2,24.737168,0.724779,49.474335,1.249985,117.206120
3,24.737168,0.925928,74.211503,2.175913,142.869216
4,24.737168,1.128667,98.948670,3.304580,168.735051
...,...,...,...,...,...
356,24.737168,387.993880,8806.431664,40652.969540,49525.883004
357,24.737168,391.254455,8831.168831,41044.223995,49941.874626
358,24.737168,394.540789,8855.905999,41438.764784,50361.152583


## Project for a Entire Holdings (Multiple Symbols)

In [7]:
# 30 years
n_months: int = 12*30
usd_zar_exchange_rate: float = 16.17
holdings: pd.DataFrame = personal_asset_data_access.get_personal_transactions()
holdings_with_profits: pd.DataFrame = asset_calculations_engine.calculate_holdings_profits(holdings=holdings)
# Simply R 1000 converted to USD and split evenly among how many assets are held.
monthly_deposits: list = [(2000 / usd_zar_exchange_rate / len(holdings_with_profits)) for h in holdings_with_profits]
holdings_growth = instance.project_monthly_holdings_growth(n_months=n_months,
                                                           holdings_with_profits=holdings_with_profits,
                                                           monthly_deposits=monthly_deposits)

holdings_growth

,month,deposits_withdrawals,interest,total_deposits_withdrawals,accrued_interest,balance
0,1,98.94867,0.022414,98.948670,6.249048e+00,3.777473e+02
1,2,98.94867,0.020871,197.897341,1.430119e+01,4.847481e+02
2,3,98.94867,0.020040,296.846011,2.421398e+01,5.936096e+02
3,4,98.94867,0.019544,395.794682,3.604687e+01,7.043912e+02
4,5,98.94867,0.019235,494.743352,4.986122e+01,8.171542e+02
...,...,...,...,...,...,...
355,356,98.94867,0.036054,35225.726654,3.730639e+08,3.730994e+08
356,357,98.94867,0.036057,35324.675325,3.870199e+08,3.870555e+08
357,358,98.94867,0.036060,35423.623995,4.014991e+08,4.015348e+08
358,359,98.94867,0.036063,35522.572665,4.165213e+08,4.165571e+08


## Project Wealth Over Months

In [8]:
monthly_zar_deposit: float = 2000
usd_zar_exchange_rate: float = 16.17
monthly_deposits: list = [(monthly_zar_deposit / usd_zar_exchange_rate / len(holdings_with_profits)) for h in holdings_with_profits]
wealth_projection = instance.project_wealth_growth(holdings_with_profits=holdings_with_profits,
                                                   monthly_deposits=monthly_deposits)

wealth_projection

,Target Balance (USD),Target Balance Eng (USD),ETA (Months),ETA (Years)
0,1.048880e+03,one thousand and forty-eight point eight eight...,6,0.500000
1,1.014091e+04,"ten thousand, one hundred and forty point nine...",51,4.250000
2,1.027070e+05,"one hundred and two thousand, seven hundred an...",124,10.333333
3,1.030030e+06,"one million, thirty thousand and thirty point ...",192,16.000000
4,1.007945e+07,"ten million, seventy-nine thousand, four hundr...",256,21.333333
5,1.034202e+08,"one hundred and three million, four hundred an...",320,26.666667
6,1.006614e+09,"one billion, six million, six hundred and thir...",382,31.833333
7,1.025766e+10,"ten billion, two hundred and fifty-seven milli...",445,37.083333
